In [8]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [10]:
!pip install rouge-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
import csv
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import torch
import numpy as np

# Define models and their corresponding prompts
models = [
    {"name": "facebook/blenderbot-400M-distill", "prompt": "What is your favorite genre of music and why?"},
    {"name": "facebook/blenderbot-3B", "prompt": "Who are some of the most influential musicians or bands in your opinion?"},
    {"name": "Pi3141/DialoGPT-medium-elon-2", "prompt": "How do you discover new music? Do you use any apps or websites?"},
    {"name": "ToddGoldfarb/Cadet-Tiny", "prompt": "How often do you listen to music and where do you listen to it?"},
    {"name": "satvikag/chatbot", "prompt": "Do you prefer to listen to music with lyrics or without lyrics? Why?"},
]

# Define generated prompts for each evaluation prompt
generated_prompts = [
    "Classical music for its soothing and timeless qualities",
    "Beethoven, Mozart, Bach, Chopin, and Tchaikovsky are influential musicians",
    "I discover new music by using Bing and YouTube.",
    "I listen to music daily, mostly on my phone",
    "I prefer music with lyrics to understand emotions and learn languages."
]

# Initialize tokenizer and model for each model
for model_info in models:
    model_info["tokenizer"] = AutoTokenizer.from_pretrained(model_info["name"])
    if model_info["name"] in ["Pi3141/DialoGPT-medium-elon-2", "satvikag/chatbot"]:
        model_info["model"] = AutoModelForCausalLM.from_pretrained(model_info["name"])
    else:
        model_info["model"] = AutoModelForSeq2SeqLM.from_pretrained(model_info["name"])

# Define evaluation metrics
def calculate_bleu(reference, candidate):
    return sentence_bleu([reference.split()], candidate.split())

def calculate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    rouge1 = scores['rouge1'].fmeasure
    rouge2 = scores['rouge2'].fmeasure
    rougeL = scores['rougeL'].fmeasure
    return rouge1, rouge2, rougeL

def calculate_response_length(text):
    return len(text.split())

# Store responses for each model
responses = []

# Generate responses for each prompt using each model
for i, model_info in enumerate(models):
    prompt_text = model_info["prompt"]
    generated_prompt = generated_prompts[i]
    responses_for_prompt = {"prompt": prompt_text}
    for model_info in models:
        tokenizer = model_info["tokenizer"]
        model = model_info["model"]
        input_ids = tokenizer(generated_prompt, return_tensors="pt").input_ids
        output = model.generate(input_ids, max_length=100, num_return_sequences=1, temperature=0.9)
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        responses_for_prompt[model_info["name"]] = response
    responses.append(responses_for_prompt)

# Initialize dictionaries to store total metrics and count
total_metrics = {model["name"]: {"BLEU": 0, "ROUGE-1": 0, "ROUGE-2": 0, "ROUGE-L": 0, "Response Length": 0} for model in models}
count = {model["name"]: 0 for model in models}

# Compare responses using evaluation metrics and calculate total metrics
for response in responses:
    for model_info in models:
        model_name = model_info["name"]
        response_text = response[model_name]
        # Get the generated prompt for evaluation
        generated_prompt = response["prompt"]
        # Evaluate response using BLEU score
        bleu_score = calculate_bleu(generated_prompt, response_text)
        # Evaluate response using ROUGE score
        rouge1, rouge2, rougeL = calculate_rouge(generated_prompt, response_text)
        # Evaluate response using response length
        response_length = calculate_response_length(response_text)
        # Update total metrics and count
        total_metrics[model_name]["BLEU"] += bleu_score
        total_metrics[model_name]["ROUGE-1"] += rouge1
        total_metrics[model_name]["ROUGE-2"] += rouge2
        total_metrics[model_name]["ROUGE-L"] += rougeL
        total_metrics[model_name]["Response Length"] += response_length
        count[model_name] += 1

# Calculate average metrics
average_metrics = {}
for model_name in total_metrics:
    average_metrics[model_name] = {metric: total_metrics[model_name][metric] / count[model_name] for metric in total_metrics[model_name]}

# Save average metrics to a CSV file
with open("evaluation_results.csv", "w", newline="") as csvfile:
    fieldnames = ["Model", "BLEU", "ROUGE-1", "ROUGE-2", "ROUGE-L", "Response Length"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for model_name, metrics in average_metrics.items():
        row = {"Model": model_name}
        row.update(metrics)
        writer.writerow(row)

print("Evaluation results saved to evaluation_results.csv")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Evaluation results saved to evaluation_results.csv


In [12]:
pip install Topsis-Kamal-102103259

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [15]:
!python -m Topsis_Kamal_102103259 evaluation_results.csv "1,1,1,1,0.5" "+,+,+,+,-" output.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
